In [ ]:
from ipywidgets import interactive, Output
from ipywidgets import Dropdown, IntSlider, SelectMultiple, Combobox, Button
from IPython.display import display

from surianalytics.connectors import RESTSciriusConnector

import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
FILTERS = {
    "default": "*",
    "tls_out_from_asset": "event_type:alert AND app_proto:tls AND asset.src_ip:* AND NOT asset.dest_ip:* AND NOT alert.signature:*SIGHTINGS*"
}

In [ ]:
c = RESTSciriusConnector()
c.set_page_size(1000)
c.set_query_delta(hours=0, minutes=60)
DF = c.get_alerts_df(qfilter=FILTERS["default"])

In [ ]:
SLIDER_TIME_MINUTES = IntSlider(
    value=60,
    min=10,
    max=180,
    step=5,
    description='Minutes',
    orientation='horizontal',
    readout=True,
)

In [ ]:
SELECT_COLUMNS = Combobox(options=sorted(list(DF.dropna(how="all", axis=1).columns.values)))

COLUMNS = []
OUTPUT_COLS = Output()
def handler_add_col(args):
    global COLUMNS
    COLUMNS.append(SELECT_COLUMNS.value)
    OUTPUT_COLS.clear_output()
    with OUTPUT_COLS:
        display(COLUMNS)

BUTTON_ADD_COL = Button(description="Add column")
BUTTON_ADD_COL.on_click(handler_add_col)

In [ ]:
DROPDOWN_FILTERS = Dropdown(
    options=list(FILTERS.keys()),
    value="default",
    description="filter"
)
def handler_select_filter(selection):
    global DF
    c.set_query_delta(minutes=SLIDER_TIME_MINUTES.value)
    DF = c.get_alerts_df(qfilter=FILTERS[selection])
    SELECT_COLUMNS.options = sorted(list(DF.dropna(how="all", axis=1).columns.values))
    
interactive(handler_select_filter, selection=DROPDOWN_FILTERS)

display(SLIDER_TIME_MINUTES)
display(DROPDOWN_FILTERS)
display(SELECT_COLUMNS)
display(BUTTON_ADD_COL)
display(OUTPUT_COLS)

In [ ]:
DF[list(COLUMNS)]

In [ ]:
DF